# STREL

In [ ]:
import torch
import torch.nn as nn

## Reach

In [ ]:
class Reach():
    """
    Reachability operator for STREL. Models bounded or unbounded reach
    over a spatial graph.
    """
    def __init__(
        self,
        distance_matrix,
        d1,
        d2,
        graph_nodes,
        is_unbounded: bool = False,
        distance_domain_min=None,
        distance_domain_max=None,
    ) -> None:
        super().__init__()
        self.d1 = d1
        self.d2 = d2
        self.is_unbounded = is_unbounded
        self.distance_domain_min = distance_domain_min
        self.distance_domain_max = distance_domain_max
        self.graph_nodes = graph_nodes

        self.weight_matrix = distance_matrix
        self.adjacency_matrix = (distance_matrix > 0).int()

        self.boolean_min_satisfaction = torch.tensor(0.0)
        self.quantitative_min_satisfaction = torch.tensor(float('-inf'))

    def __call__(self, s1, s2):
        return self._boolean(s1, s2)
        '''
        if quantitative:
            return self._quantitative(s1, s2)
        else:
            return self._boolean(s1, s2)
        '''

    def __str__(self) -> str:
        bound_type = "unbounded" if self.is_unbounded else f"[{self.d1},{self.d2}]"
        return f"Reach{bound_type}"

    def time_depth(self) -> int:
        return 0

    def neighbors_fn(self, node):
        mask = self.adjacency_matrix[:, node] > 0
        neighbors = self.graph_nodes[mask]
        neigh_pairs = [(i.item(), self.weight_matrix[i, node].item()) for i in neighbors]
        print('node = ', node, ' has neigh_pairs = ', neigh_pairs)
        return neigh_pairs

    def distance_function(self, weight):
        return weight

    # ----------------
    #    Boolean
    # ----------------

    def _boolean(self, s1, s2):
        if self.is_unbounded:
            return self._unbounded_reach_boolean(s1, s2)
        else:
            return self._bounded_reach_boolean(s1, s2)

    def _bounded_reach_boolean(self, s1, s2):
        s = torch.zeros(len(self.graph_nodes), requires_grad=True)

        for i, lt in enumerate(self.graph_nodes):
            l = lt.item()
            if self.d1 == self.distance_domain_min:
                s = s.clone().scatter_(0, torch.tensor([l]), s2[l])
            else:
                s = s.clone().scatter_(0, torch.tensor([l]), self.boolean_min_satisfaction)

        Q = {llt.item(): [(s2[llt.item()], self.distance_domain_min)] for llt in self.graph_nodes}

        while Q:
            print('Q = ', Q)
            Q_prime = {}
            for l in Q.keys():
                print('l = ', l)
                for v, d in Q[l]:
                    for l_prime, w in self.neighbors_fn(l):
                        print(f'neigh of {l}: {l_prime}')
                        v_new = torch.minimum(v, s1[l_prime])
                        d_new = d + w

                        if self.d1 <= d_new <= self.d2:
                            current_val = s[l_prime]
                            new_val = torch.maximum(current_val, v_new)
                            s = s.clone().scatter_(0, torch.tensor([l_prime]), new_val)
                            print('s = ', s)

                        if d_new < self.d2:

                            existing_entries = Q_prime.get(l_prime, [])
                            updated = False
                            new_entries = []
                            for vv, dd in existing_entries:
                                if dd == d_new:
                                    new_v = torch.maximum(vv, v_new)
                                    new_entries.append((new_v, dd))
                                    updated = True
                                else:
                                    new_entries.append((vv, dd))

                            if not updated:
                                new_entries.append((v_new, d_new))
                            Q_prime[l_prime] = new_entries

            print('Q_prime = ', Q_prime)
            Q = Q_prime
        return s

    def _unbounded_reach_boolean(self, s1, s2):

        if self.d1 == self.distance_domain_min:
            s = s2
        else:
            d_max = torch.max(self.distance_function(self.weight_matrix))
            self.d2 = self.d1 + d_max
            s = self._bounded_reach_boolean(s1, s2)

        T = [n for n in self.graph_nodes]

        while T:
            T_prime = []

            for l in T:
                for l_prime, w in self.neighbors_fn(l):
                    v_prime = torch.minimum(s[l], s1[l_prime])
                    v_prime = torch.maximum(v_prime, s[l_prime])

                    if v_prime != s[l_prime]:
                        s = s.clone().scatter_(0, torch.tensor([l_prime]), v_prime)
                        T_prime.append(l_prime)

            T = T_prime

        return s

    # ---------------------
    #     Quantitative
    # ---------------------

    def _quantitative(self, s1, s2):
        if self.is_unbounded:
            return self._unbounded_reach_quantitative(s1, s2)
        else:
            return self._bounded_reach_quantitative(s1, s2)

    def _bounded_reach_quantitative(self, s1, s2):
        s = torch.full((len(self.graph_nodes),), self.quantitative_min_satisfaction, requires_grad=True)

        for i, lt in enumerate(self.graph_nodes):
            l = lt.item()
            if self.d1 == self.distance_domain_min:
                s = s.clone().scatter_(0, torch.tensor([l]), s2[l])
            else:
                s = s.clone().scatter_(0, torch.tensor([l]), self.quantitative_min_satisfaction)

        Q = {
            llt.item(): [(s2[llt.item()], self.distance_domain_min)]
            for llt in self.graph_nodes
        }

        while Q:
            Q_prime = {}
            for l in Q.keys():
                for (v, d) in Q[l]:
                    for l_prime, w in self.neighbors_fn(l):
                        v_new = torch.minimum(v, s1[l_prime])
                        d_new = d + w

                        if self.d1 <= d_new <= self.d2:
                            current_val = s[l_prime]
                            new_val = torch.maximum(current_val, v_new)
                            s = s.clone().scatter_(0, torch.tensor([l_prime]), new_val)

                        if d_new < self.d2:
                            existing_entries = Q_prime.get(l_prime, [])
                            updated = False
                            new_entries = []
                            for (vv, dd) in existing_entries:
                                if dd == d_new:
                                    new_v = torch.maximum(vv, v_new)
                                    new_entries.append((new_v, dd))
                                    updated = True
                                else:
                                    new_entries.append((vv, dd))
                            if not updated:
                                new_entries.append((v_new, d_new))
                            Q_prime[l_prime] = new_entries
            Q = Q_prime

        return s

    def _unbounded_reach_quantitative(self, s1, s2):

        if self.d1 == self.distance_domain_min:
            s = s2
        else:
            d_max = torch.max(self.distance_function(self.weight_matrix))
            self.d2 = self.d1 + d_max
            s = self._bounded_reach_quantitative(s1, s2)

        T = [n for n in self.graph_nodes]

        while T:
            T_prime = []

            for l in T:
                for l_prime, w in self.neighbors_fn(l):
                    v_prime = torch.minimum(s[l], s1[l_prime])
                    new_val = torch.maximum(v_prime, s[l_prime])
                    if new_val != s[l_prime]:
                        s = s.clone().scatter_(0, torch.tensor([l_prime]), new_val)
                        T_prime.append(l_prime)
            T = T_prime

        return s

## Escape

In [ ]:
class Escape():
    """
    Escape operator for STREL. Models escape condition over a spatial graph.
    """
    def __init__(
        self,
        distance_matrix,
        d1,
        d2,
        graph_nodes,
        distance_domain_min=None,
        distance_domain_max=None,
    ) -> None:
        self.d1 = d1
        self.d2 = d2
        self.distance_domain_min = distance_domain_min
        self.distance_domain_max = distance_domain_max
        self.graph_nodes = graph_nodes

        self.weight_matrix = distance_matrix
        self.adjacency_matrix = (distance_matrix > 0).int()

        self.boolean_min_satisfaction = torch.tensor(0.0)
        self.quantitative_min_satisfaction = torch.tensor(float('-inf'))

    def __call__(self, s1, quantitative=False):
        return self._quantitative(s1) if quantitative else self._boolean(s1)

    def time_depth(self) -> int:
        return 0

    def neighbors_fn(self, node):
        mask = (self.adjacency_matrix[:, node] > 0)
        neighbors = self.graph_nodes[mask]
        return [(i.item(), self.weight_matrix[i, node].item()) for i in neighbors]

    def forward_neighbors_fn(self, node):
        mask = (self.adjacency_matrix[node, :] > 0)
        neighbors = self.graph_nodes[mask]
        return [(j.item(), self.weight_matrix[node, j].item()) for j in neighbors]

    def compute_min_distance_matrix(self):

        n = len(self.graph_nodes) # Numero di nodi nel grafo
        D = torch.full((n, n), float('inf')) # matrice n x n inizializzata con infinito (inf). Alla fine conterrà le distanze minime tra ogni coppia di nodi.

        for start in range(n): # Calcola le distanze dal nodo start a tutti gli altri
            visited = torch.zeros(n, dtype=torch.bool) # tiene traccia dei nodi già visitati
            distance = torch.full((n,), float('inf')) # array delle distanze minime dal nodo start
            distance[start] = 0 # Inizia da 'start'

            frontier = torch.zeros(n, dtype=torch.bool) # Nodi da visitare nel livello corrente
            frontier[start] = True # Inizia da 'start'

            while frontier.any(): # itera finché ci sono nodi nella frontiera
                next_frontier = torch.zeros(n, dtype=torch.bool) # Prepara la frontier per il livello successivo

                for node in torch.nonzero(frontier).flatten(): # Per ogni nodo nella frontiera attuale, lo segna come visitato
                    node = node.item()
                    visited[node] = True

                    for neighbor, weight in self.forward_neighbors_fn(node): # Recupera i vicini del nodo e il peso del collegamento
                        if visited[neighbor]:
                            continue
                        # edge_cost = weight if getattr(self, "use_weights", False) else 1.0 # Se use_weights=True, usa il peso reale dell’arco. Altrimenti, ogni arco conta 1 (hop count booleano)
                        edge_cost = weight # if self.use_weights else 1.0
                        new_dist = distance[node] + edge_cost # Calcola la distanza cumulativa provvisoria al vicino

                        # Se è una distanza più breve di quella attuale, la aggiorna e aggiunge il vicino alla prossima frontiera
                        if new_dist < distance[neighbor]:
                            distance[neighbor] = new_dist
                            next_frontier[neighbor] = True

                frontier = next_frontier # Passa alla prossima frontiera da esplorare

            D[start] = distance # Dopo aver esplorato tutti i percorsi dal nodo start, salva le distanze finali

        return D # Restituisce la matrice completa D con le distanze minime tra tutti i nodi, pesate o in hop

    # ----------------
    #    Boolean
    # ----------------

    def _boolean(self, s1):
        L = self.graph_nodes
        n = len(L)

        D = self.compute_min_distance_matrix()

        e = torch.ones((n, n), requires_grad=True) * self.boolean_min_satisfaction
        e = e - torch.diag(torch.diag(e)) + torch.diag(s1)

        T = [(i, i) for i in range(n)]

        while T:
            T_prime = []
            e_prime = e.clone()

            for l1, l2 in T:
                for l1_prime, w in self.neighbors_fn(l1):
                    new_val = torch.minimum(s1[l1_prime], e[l1, l2])
                    old_val = e[l1_prime, l2]
                    combined = torch.maximum(old_val, new_val)

                    if combined != old_val:
                        e_prime = e_prime.clone().index_put_(tuple(torch.tensor([[l1_prime], [l2]])), combined)
                        T_prime.append((l1_prime, l2))

            T = T_prime
            e = e_prime

        s = torch.ones(n, requires_grad=True) * self.boolean_min_satisfaction
        for i in range(n):
            vals = [
                e[i, j] for j in range(n)
                if self.d1 <= D[i, j] <= self.d2
            ]
            if vals:
                max_val = torch.stack(vals).max()
                s = s.clone().scatter_(0, torch.tensor([i]), max_val.unsqueeze(0))

        return s

    # ---------------------
    #     Quantitative
    # ---------------------

    def _quantitative(self, s1):
        L = self.graph_nodes
        n = len(L)

        D = self.compute_min_distance_matrix()

        e = torch.ones((n, n), requires_grad=True) * self.quantitative_min_satisfaction
        e = e - torch.diag(torch.diag(e)) + torch.diag(s1)

        T = [(i, i) for i in range(n)]

        while T:
            T_prime = []
            e_prime = e.clone()

            for l1, l2 in T:
                for l1_prime, w in self.neighbors_fn(l1):
                    new_val = torch.minimum(s1[l1_prime], e[l1, l2])
                    old_val = e[l1_prime, l2]
                    combined = torch.maximum(old_val, new_val)

                    if combined != old_val:
                        e_prime = e_prime.clone().index_put_(
                            tuple(torch.tensor([[l1_prime], [l2]])), combined
                        )
                        T_prime.append((l1_prime, l2))

            T = T_prime
            e = e_prime

        s = torch.ones(n, requires_grad=True) * self.quantitative_min_satisfaction
        for i in range(n):
            vals = [
                e[i, j] for j in range(n)
                if self.d1 <= D[i, j] <= self.d2
            ]
            if vals:
                max_val = torch.stack(vals).max()
                s = s.clone().scatter_(0, torch.tensor([i]), max_val.unsqueeze(0))

        return s

## Somewhere

In [ ]:
class Somewhere():
    """
    Somewhere operator for STREL. Models existence of a satisfying location within a distance interval.
    """
    def __init__(
        self,
        distance_matrix,
        d1,
        d2,
        graph_nodes,
        distance_domain_min=None,
        distance_domain_max=None,
    ) -> None:
        self.d1 = d1
        self.d2 = d2
        self.distance_domain_min = distance_domain_min
        self.distance_domain_max = distance_domain_max
        self.graph_nodes = graph_nodes

        self.reach_op = Reach(
            distance_matrix=distance_matrix,
            d1=d1,
            d2=d2,
            graph_nodes=graph_nodes,
            distance_domain_min=distance_domain_min,
            distance_domain_max=distance_domain_max
        )

        self.boolean_min_satisfaction = torch.tensor(0.0)
        self.quantitative_min_satisfaction = torch.tensor(float('-inf'))

    def __call__(self, phi, quantitative=False):
        if quantitative:
            return self._quantitative(phi)
        else:
            return self._boolean(phi)

    def __str__(self) -> str:
        return f"Somewhere^f_[{self.d1},{self.d2}]"

    def time_depth(self) -> int:
        return 0

    # ----------------
    #    Boolean
    # ----------------

    def _boolean(self, phi):
        # somewhere^f_[0, d] φ = true R^f_[0, d] φ
        s1 = torch.ones(len(self.graph_nodes), requires_grad=True)
        return self.reach_op._boolean(s1, phi)

    # ---------------------
    #     Quantitative
    # ---------------------

    def _quantitative(self, phi):
        # somewhere^f_[0, d] φ = true R^f_[0, d] φ
        s1 = torch.ones(len(self.graph_nodes), requires_grad=True) * float('inf')
        return self.reach_op._quantitative(s1, phi)

## Everywhere

In [ ]:
class Everywhere():
    """
    Everywhere operator for STREL. Models satisfaction of a property at all locations within a distance interval.
    """
    def __init__(
        self,
        distance_matrix,
        d1,
        d2,
        graph_nodes,
        distance_domain_min=None,
        distance_domain_max=None,
    ) -> None:
        self.d1 = d1
        self.d2 = d2
        self.distance_domain_min = distance_domain_min
        self.distance_domain_max = distance_domain_max
        self.graph_nodes = graph_nodes

        self.somewhere_op = Somewhere(
            distance_matrix=distance_matrix,
            d1=d1,
            d2=d2,
            graph_nodes=graph_nodes,
            distance_domain_min=distance_domain_min,
            distance_domain_max=distance_domain_max
        )

        self.boolean_min_satisfaction = torch.tensor(0.0)
        self.quantitative_min_satisfaction = torch.tensor(float('-inf'))

    def __call__(self, phi, quantitative=False):
        if quantitative:
            return self._quantitative(phi)
        else:
            return self._boolean(phi)

    def __str__(self) -> str:
        return f"Everywhere^f_[{self.d1},{self.d2}]"

    def time_depth(self) -> int:
        return 0

    # ----------------
    #    Boolean
    # ----------------

    def _boolean(self, phi):
        # everywhere^f_[0, d] φ = ¬somewhere^f_[0, d] ¬φ
        neg_phi = 1 - phi
        somewhere_neg_phi = self.somewhere_op._boolean(neg_phi)
        return 1 - somewhere_neg_phi

    # ---------------------
    #     Quantitative
    # ---------------------

    def _quantitative(self, phi):
        # everywhere^f_[0, d] φ = ¬somewhere^f_[0, d] ¬φ
        neg_phi = -phi
        somewhere_neg_phi = self.somewhere_op._quantitative(neg_phi)
        return -somewhere_neg_phi

## Sorround

In [ ]:
class Surround():
    """
    Surround operator for STREL. Models being surrounded by φ2 while in φ1 with distance constraints.
    """
    def __init__(
        self,
        distance_matrix,
        d1,
        d2,
        graph_nodes,
        distance_domain_min=None,
        distance_domain_max=None,
    ) -> None:
        self.d1 = d1
        self.d2 = d2
        self.distance_domain_min = distance_domain_min
        self.distance_domain_max = distance_domain_max
        self.graph_nodes = graph_nodes

        # Reach operator for [0,d] interval
        self.reach_op = Reach(
            distance_matrix=distance_matrix,
            d1=0,
            d2=d2,
            graph_nodes=graph_nodes,
            distance_domain_min=distance_domain_min,
            distance_domain_max=distance_domain_max
        )

        # Escape operator for [the negated interval ¬[d,∞] (which is [0,d))
        self.escape_op = Escape(
            distance_matrix=distance_matrix,
            d1=distance_domain_min,  # 0
            d2=d1,  # d (exclusive upper bound)
            # d1=d1,  # Start from d
            # d2=distance_domain_max,  # Effectively ∞
            graph_nodes=graph_nodes,
            distance_domain_min=distance_domain_min,
            distance_domain_max=distance_domain_max
        )

        self.boolean_min_satisfaction = torch.tensor(0.0)
        self.quantitative_min_satisfaction = torch.tensor(float('-inf'))

    def __call__(self, phi1, phi2, quantitative=False):
        if quantitative:
            return self._quantitative(phi1, phi2)
        else:
            return self._boolean(phi1, phi2)

    def __str__(self) -> str:
        return f"Surround^f_[{self.d1},{self.d2}]"

    def time_depth(self) -> int:
        return 0

    # ----------------
    #    Boolean
    # ----------------

    def _boolean(self, phi1, phi2):
        # φ1 sorround^f_[0, d] φ2 = φ1 ∧ ¬(φ1 R^f_[0, d] ¬(φ1 ∨ φ2)) ∧ ¬(E^f_[d,∞] φ1)

        # First part: φ1
        part1 = phi1

        # Second part: ¬(φ1 R^f_[0, d] ¬(φ1 ∨ φ2))
        phi1_or_phi2 = torch.maximum(phi1, phi2)
        neg_phi1_or_phi2 = 1 - phi1_or_phi2
        reach_part = self.reach_op._boolean(phi1, neg_phi1_or_phi2)
        part2 = 1 - reach_part

        # Third part: ¬(E^f_¬[d,∞] φ1)
        escape_part = self.escape_op._boolean(phi1)
        part3 = 1 - escape_part

        # Combine all parts
        result = torch.minimum(part1, torch.minimum(part2, part3))
        return result

    # ---------------------
    #     Quantitative
    # ---------------------

    def _quantitative(self, phi1, phi2):
        # φ1 sorround^f_[0, d] φ2 = φ1 ∧ ¬(φ1 R^f_[0, d] ¬(φ1 ∨ φ2)) ∧ ¬(E^f_[d,∞] φ1)

        # First part: φ1
        part1 = phi1

        # Second part: ¬(φ1 R^f_[0, d] ¬(φ1 ∨ φ2))
        phi1_or_phi2 = torch.maximum(phi1, phi2)
        neg_phi1_or_phi2 = -phi1_or_phi2
        reach_part = self.reach_op._quantitative(phi1, neg_phi1_or_phi2)
        part2 = -reach_part

        # Third part: ¬(E^f_¬[d,∞] φ1)
        escape_part = self.escape_op._quantitative(phi1)
        part3 = -escape_part

        # Combine all parts
        result = torch.minimum(part1, torch.minimum(part2, part3))
        return result

# Testing